In [1]:
simbolo_test = "BTG"
simbolo_validazione = "DHT"
n_simboli_addestramento = 2000
epochs = 100
batch_size = 16
optimizer = "adam"
#learning_rate = 0.001
n_timesteps = 60 # n. barre del periodo passato per la ricerca di pattern, inclusa ultima data disponibile
giorni_previsione = 10 # n. barre nel futuro di cui si desidera prevedere il prezzo

elenco_features = [
    "Close",
    "EMA_5", 
    "EMA_20", 
    "EMA_50",
    "Open",  
    "High",
    "Low",
    "Volume",
    "MACDh",
    "PSAR",
    "PSARaf",
    "SUPERT", 
    "TRIX",
    "ATR",
    "DM_OSC",
    "ADX",
    "CDL_2CROWS", "CDL_3BLACKCROWS", "CDL_3INSIDE", "CDL_3LINESTRIKE", "CDL_3OUTSIDE", "CDL_3STARSINSOUTH", "CDL_3WHITESOLDIERS", "CDL_ABANDONEDBABY", "CDL_ADVANCEBLOCK", "CDL_BELTHOLD", "CDL_BREAKAWAY", "CDL_CLOSINGMARUBOZU", "CDL_CONCEALBABYSWALL", "CDL_COUNTERATTACK", "CDL_DARKCLOUDCOVER", "CDL_DOJI_10_0.1", "CDL_DOJISTAR", "CDL_DRAGONFLYDOJI", "CDL_ENGULFING", "CDL_EVENINGDOJISTAR", "CDL_EVENINGSTAR", "CDL_GAPSIDESIDEWHITE", "CDL_GRAVESTONEDOJI", "CDL_HAMMER", "CDL_HANGINGMAN", "CDL_HARAMI", "CDL_HARAMICROSS", "CDL_HIGHWAVE", "CDL_HIKKAKE", "CDL_HIKKAKEMOD", "CDL_HOMINGPIGEON", "CDL_IDENTICAL3CROWS", "CDL_INNECK", "CDL_INSIDE", "CDL_INVERTEDHAMMER", "CDL_KICKING", "CDL_KICKINGBYLENGTH", "CDL_LADDERBOTTOM", "CDL_LONGLEGGEDDOJI", "CDL_LONGLINE", "CDL_MARUBOZU", "CDL_MATCHINGLOW", "CDL_MATHOLD", "CDL_MORNINGDOJISTAR", "CDL_MORNINGSTAR", "CDL_ONNECK", "CDL_PIERCING", "CDL_RICKSHAWMAN", "CDL_RISEFALL3METHODS", "CDL_SEPARATINGLINES", "CDL_SHOOTINGSTAR", "CDL_SHORTLINE", "CDL_SPINNINGTOP", "CDL_STALLEDPATTERN", "CDL_STICKSANDWICH", "CDL_TAKURI", "CDL_TASUKIGAP", "CDL_THRUSTING", "CDL_TRISTAR", "CDL_UNIQUE3RIVER", "CDL_UPSIDEGAP2CROWS", "CDL_XSIDEGAP3METHODS",
]

elenco_targets = [
    "EMA_5",
    "EMA_20", 
    "EMA_50"    
]

col_features = {col: idx for idx, col in enumerate(elenco_features)}
col_targets = {col: idx for idx, col in enumerate(elenco_targets)}
n_features = len(col_features)
n_targets = len(col_targets) 
    

def crea_modello(n_timesteps, giorni_previsione, n_targets, n_features):
        model = Sequential()

        # Layer LSTM iniziale
        model.add(LSTM(70, input_shape=(n_timesteps, n_features)))
        model.add(RepeatVector(giorni_previsione))

        # lstm-2
        model.add(LSTM(90, return_sequences=True,
                       kernel_regularizer=l2(4.153520011432108e-05)))
        model.add(Dropout(0.3))
        # lstm-3
        model.add(LSTM(130, return_sequences=True,
                       kernel_regularizer=l2(4.153520011432108e-05)))
        model.add(Dropout(0.3))
        # lstm-4
        model.add(LSTM(70, return_sequences=True,
                       kernel_regularizer=l2(4.153520011432108e-05)))
        model.add(Dropout(0.4))

        # Aggiunta di layer Dense
        model.add(TimeDistributed(Dense(70, activation='relu')))
        model.add(Dropout(0))

        model.add(TimeDistributed(Dense(n_targets, activation='linear')))

        model.compile(optimizer="adam",
                      loss='mean_absolute_percentage_error')

        return model

#from sklearn.preprocessing import PowerTransformer
#from sklearn.preprocessing import RobustScaler
#from sklearn.base import clone
#scaler = PowerTransformer()
#scaler = RobustScaler()



In [2]:
print("Importa librerie", end="", flush=True)

#from keras.losses import Huber
#print(".", end="", flush=True)
import pandas as pd
print(".", end="", flush=True)
import yfinance as yf
print(".", end="", flush=True)
import numpy as np
print(".", end="", flush=True)
import tensorflow as tf
print(".", end="", flush=True)
import funzioni as fx
print(".", end="", flush=True)
from tensorflow.keras.layers import Dropout
print(".", end="", flush=True)
from tensorflow.keras.layers import RepeatVector
print(".", end="", flush=True)
from tensorflow.keras.layers import TimeDistributed
print(".", end="", flush=True)
from keras.regularizers import l2
print(".", end="", flush=True)
from tensorflow.keras import Sequential
print(".", end="", flush=True)
from tensorflow.keras.layers import LSTM, Dense
print(".", end="", flush=True)
#from tensorflow.keras.optimizers import Adam
#print(".", end="", flush=True)
#from sklearn.model_selection import train_test_split
#print(".", end="", flush=True)
from tensorflow.keras.models import load_model
print(".", end="", flush=True)
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
print(".", end="", flush=True)
import os
print("=", end="\n", flush=True)

print("Download lista ticker")
lista_ticker = pd.read_parquet("Tickers_De_Giro.parquet")
lista_ticker = lista_ticker.sample(frac=1).reset_index(drop=True)
lista_ticker = lista_ticker.loc[(lista_ticker["Ticker"] != simbolo_test) & (lista_ticker["Ticker"] != simbolo_validazione) & (lista_ticker["Categoria"] != "D"), :]

if os.path.exists('LSTM.keras'):
    print("Caricamento modello esistente")
    model = load_model("LSTM.keras")
else:
    print("Creazione modello")
    model = crea_modello(n_timesteps, giorni_previsione, n_targets, n_features)


Importa librerie.............=
Download lista ticker
Caricamento modello esistente


In [3]:
print("Download dati ticker validazione")
ticker_val = yf.download(simbolo_validazione, start='2010-01-01', end='2023-12-31', progress=False)
ticker_val.index = ticker_val.index.date
print("Calcolo indicatori ticker validazione")
ticker_val = fx.crea_indicatori(ticker_val)
ticker_val.dropna(axis=0, inplace=True)

idx_val, X_val, Y_val = fx.to_XY(ticker_val, elenco_features, elenco_targets, n_timesteps, giorni_previsione, addestramento=True)
X_val, _ = fx.scala(X_val)
Y_val, _ = fx.scala(Y_val)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

for i_ticker in range (n_simboli_addestramento + 1):
    nome_simbolo = lista_ticker["Ticker"].iloc[i_ticker]
    print(f"\033[48;5;42m{i_ticker+1} di {n_simboli_addestramento}: Ticker {nome_simbolo}\033[0m")
    print("Download dati ticker")
    try:
        ticker = yf.download(nome_simbolo, start='2010-01-01', end='2023-12-31', progress=False)
        if ticker["Close"].iloc[-1] >= 1:
            ticker.index = ticker.index.date
            print("Calcolo indicatori ticker")
            ticker = fx.crea_indicatori(ticker)
            ticker.dropna(axis=0, inplace=True)

            print("Definizione features e target")
            idx, X, Y = fx.to_XY(ticker, elenco_features, elenco_targets, n_timesteps, giorni_previsione, addestramento=True)

            print("Scaler")
            X, X_scaler = fx.scala(X)
            Y, Y_scaler = fx.scala(Y)

            print(f"Addestramento modello epochs={epochs}, batch_size={batch_size}")
            model.fit(X, Y, epochs=epochs, batch_size=batch_size, validation_data=(X_val, Y_val), callbacks=[early_stopping, reduce_lr])
            model.save("LSTM.keras")
    except Exception as e:
        print(e)
        continue


Download dati ticker validazione
Calcolo indicatori ticker validazione
1 di 2000: Ticker SVM
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Scaler
Addestramento modello epochs=100, batch_size=16
Epoch 1/100
in user code:

    File "C:\Users\932197\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\932197\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\932197\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\932197\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\932197\AppData\Loca


KeyboardInterrupt



In [ ]:

print("Download dati ticker per previsione")
dati_previsione = yf.download(simbolo_test, start='2010-01-01', end='2023-12-31', progress=False)
dati_previsione.index = dati_previsione.index.date
dati_previsione = fx.crea_indicatori(dati_previsione)
dati_previsione.iloc[:100] = dati_previsione.iloc[:100].dropna(axis=0)

indice_prev, X_prev, Y_prev = fx.to_XY(dati_previsione, elenco_features, elenco_targets, n_timesteps, giorni_previsione, addestramento=False)

X_prev, X_scaler_prev = fx.scala(X_prev)
Y_prev, Y_scaler_prev = fx.scala(Y_prev)

pred = model.predict(X_prev)
#pred_loss, pred_mae = model.evaluate(X_prev, Y_prev)

X_prev = fx.scala_inversa(X_prev, X_scaler_prev)
pred = fx.scala_inversa(pred, Y_scaler_prev)

col_analisi = "EMA_5"

import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.subplots as sp
import yfinance as yf

def grafico1(dati_previsione, pred):
    dati_previsione = dati_previsione.loc[indice_prev]
    new_dates = pd.bdate_range(start=dati_previsione.index[-1] + pd.Timedelta(days=1), periods=giorni_previsione)
    df_new = pd.DataFrame(index=new_dates.date)
    dati_previsione = pd.concat([dati_previsione, df_new])
    risultato = pd.DataFrame({"Previsione": pred[:, giorni_previsione-1, col_targets[col_analisi]].round(2)}, index=dati_previsione.index[giorni_previsione:])
    risultato = pd.concat([dati_previsione[col_analisi], risultato], axis=1)
    
    target = go.Scatter(
        x = risultato.index,
        y = risultato[col_analisi],
        mode = 'lines',
        line = dict(color='rgba(0, 0, 0, .9)'),
        name = col_analisi
    )

    previsione = go.Scatter(
        x = risultato.index,
        y = risultato['Previsione'],
        mode = 'lines',
        line = dict(color='rgba(0, 0, 250, .9)'),
        name = 'Previsione'
    )
    '''
    err_meno = go.Scatter(
        x = risultato.index,
        y = risultato['Previsione'] - err, 
        mode = 'lines',
        line = dict(
            color='rgba(0, 0, 250, .9)',
            width=1
        ),
        connectgaps = False,
        name = 'err-'
    )                
    err_piu = go.Scatter(
        x = risultato.index,
        y = risultato['Previsione'] + err, 
        mode = 'lines',
        line = dict(
            color='rgba(0, 0, 250, .9)',
            width=1
        ),
        fill='tonexty',
        fillcolor='rgba(0, 0, 250, .2)', # Puoi cambiare il valore di alpha per regolare la trasparenza
        connectgaps = False,
        name = 'err+'
    )
    '''
    layout = dict(xaxis = dict(autorange=True),
                  yaxis = dict(title = 'Previsioni', autorange=True),
                  autosize = True,
                  margin = go.layout.Margin(
                      l=0,  # Sinistra
                      r=0,  # Destra
                      b=0,  # Basso
                      t=50,  # Alto
                      pad=0  # Padding
                  ),
                  legend = dict(traceorder = 'normal', bordercolor = 'black')
    )
    fig = sp.make_subplots(rows=1, cols=1, shared_xaxes=True)
    fig.update_layout(layout)
    fig.add_trace(target, row=1, col=1)
    fig.add_trace(previsione, row=1, col=1)
    #fig.add_trace(err_meno, row=1, col=1)
    #fig.add_trace(err_piu, row=1, col=1)
    pyo.plot(fig, filename="grafico1.html", auto_open=True)




In [ ]:
def grafico2(dati_previsione, pred):
    dati2 = dati_previsione.dropna(axis=0).iloc[-60:]
    new_dates = pd.bdate_range(start=dati2.index[-1] + pd.Timedelta(days=1), periods=giorni_previsione)
    df_new = pd.DataFrame(index=new_dates.date)

    val_prev2 = (pred[-1, i, col_targets[col_analisi]] for i in range(giorni_previsione))
    prev2 = pd.DataFrame(val_prev2, columns=["Previsione"], index=df_new.index)
    riga_iniziale = pd.DataFrame({"Previsione": [dati2[col_analisi].iloc[-1]]}, index=[dati2.index[-1]])
    prev2 = pd.concat([riga_iniziale, prev2], axis=0)
    
    target = go.Scatter(
        x = dati2.index,
        y = dati2[col_analisi].round(2),
        mode = 'lines',
        line = dict(color='rgba(0, 0, 0, .9)'),
        name = col_analisi
    )

    previsione = go.Scatter(
        x = prev2.index,
        y = prev2['Previsione'].round(2),
        mode = 'lines',
        line = dict(color='rgba(0, 0, 250, .9)'),
        name = 'Previsione'
    )
    layout = dict(xaxis = dict(autorange=True),
                  yaxis = dict(title = 'Previsioni', autorange=True),
                  autosize = True,
                  margin = go.layout.Margin(
                      l=0,  # Sinistra
                      r=0,  # Destra
                      b=0,  # Basso
                      t=50,  # Alto
                      pad=0  # Padding
                  ),
                  legend = dict(traceorder = 'normal', bordercolor = 'black')
    )
    fig = sp.make_subplots(rows=1, cols=1, shared_xaxes=True)
    fig.update_layout(layout)
    fig.add_trace(target, row=1, col=1)
    fig.add_trace(previsione, row=1, col=1)
    pyo.plot(fig, filename="grafico2.html", auto_open=True)



In [ ]:
grafico1(dati_previsione, pred)

In [ ]:
grafico2(dati_previsione, pred)